In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

import sqlite3

In [2]:
conn = sqlite3.connect('/Users/aaronlevi/Documents/sql_db/chords_list.db')
cur = conn.cursor()

In [3]:
# dfAllSongs = pd.read_csv('playlist_df.csv')
# dfAllSongs.head()

dfAllSongs = pd.DataFrame(columns = ['Song', 'Artist', 'Chords', 'Label']) 
dfAllSongs 

,Song,Artist,Chords,Label


In [5]:
chordPath  = '/Users/aaronlevi/Documents/Insight2020/chordRec/ug_chords/'
chordFiles = [f for f in listdir(chordPath) if isfile(join(chordPath, f))]

songs_db = pd.read_sql_query("SELECT Song FROM basic_info", conn)
# SELECT count(*) FROM foos WHERE bar = 'baz'

# for every song in the directory...
for iSong in range(0, len(chordFiles)):
    dfThisSong = pd.read_csv(chordPath+chordFiles[iSong])
    
    # ...check if it's already in the db, and add it to the db if not
    if sum(songs_db['Song'].str.contains( str(dfThisSong['Song'][0]) )) is 0:
        dfAllSongs = pd.concat([dfAllSongs, dfThisSong], axis=0)

In [6]:
dfAllSongs = dfAllSongs.reset_index(drop=True)
dfAllSongs.head()

,Song,Artist,Chords,Label
0,better-days,strung-out,"C/E,F/A,Bb,Gm,F,C,Am,Dm",advanced
1,mary-had-a-little-lamb,paul-mccartney,"Am7,Bb,F,E7,Am,Gm7,G#dim,F7,D7,C#,A,E,C7,G#,Eb...",advanced
2,rhiannon,fleetwood-mac,"F,C,Am",novice
3,let-it-be-me,the-everly-brothers,"F#m,A,E,C,Am,Bm,D,Em,C#m,G,B7",intermediate
4,grace-kelly,mika,"Am7,D/F#,Dsus4,C,D7,Cm7,F,A7/C#,D,Em,Dm7,D7sus...",intermediate


In [9]:
# add these songs to the sql db
dfAllSongs.to_sql('basic_info', con=conn, if_exists='append')

In [8]:
# 1331 songs in batch 1, 1099 new ones in batch 2
# print(len(dfAllSongs))

# new_songs_db = pd.read_sql_query("SELECT Song FROM basic_info", conn)
# new_songs_db['Song']

In [13]:
# now do the same for the features table in the db
# start by getting spotify credentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id= '6426e18767ac42d78a9c08f41cacba65',
                                                     client_secret= '4a2cdab50e1c4ecd9fc62cae20664e29')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [15]:
# and set up the "standard chord list" for comparison later
# put together a list of various standard major & minor chords
sl1 = ['A', 'Bb', 'B', 'C', 'C#', 'Db', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'Ab' ]
sl2 = [s + 'maj' for s in sl1]
sl3 = [s + 'm' for s in sl1]
sl4 = [s + 'min' for s in sl1]

standardList = sl1+sl2+sl3+sl4


In [26]:
# preallocate 
nUniqueChords   = []
nHardChords     = []
chordDensity    = []
difficultyLabel = []

spotFeatures = {}

# spotFeatures['key'] = []
spotFeatures['tempo'] = []
# spotFeatures['duration_ms'] = []
spotFeatures['duration_m'] = []
spotFeatures['time_signature'] = []
spotFeatures['energy'] = []

# loop throiugh dfAllSongs and get spotify info
# this is only new songs
for iSong in range(0, len(dfAllSongs)):
    songName   = str(dfAllSongs['Song'][iSong]).replace('-', ' ').lower()
    artistName = str(dfAllSongs['Artist'][iSong]).replace('-', ' ').lower()

    # search spotify api
    result = sp.search(songName, limit=10) 
    nItems = len(result['tracks']['items'])

    # if our spotify search returns anything, extract desired vars
    if nItems>0:
        # match artist info, and get URI    
        foundMatch = [] # set up a counter...
        for iArtist in range(0, nItems):    
            if result['tracks']['items'][iArtist]['artists'][0]['name'].lower == artistName:
                songURI = result['tracks']['items'][iArtist]['uri']
                foundMatch.append(True)
            else:
                foundMatch.append(False) #... that keeps track of if we found a matching artist

        # if we didn't, let's assume the first artist was the correct one, and this was a 
        # problem with spelling/special chars/case sensitivity...
        if sum(foundMatch)==0:
            songURI = result['tracks']['items'][0]['uri']

        # pull audio features for a songURI
        features = sp.audio_features(songURI)

    #     spotFeatures['key'].append(features[0]['key'])
        spotFeatures['tempo'].append(features[0]['tempo'])
        spotFeatures['duration_m'].append((features[0]['duration_ms']/1000)/60)
        spotFeatures['time_signature'].append(features[0]['time_signature'])
        spotFeatures['energy'].append(features[0]['energy'])

    # if not, fill with NaNs
    else:
        spotFeatures['tempo'].append(float('nan'))
        spotFeatures['duration_m'].append(float('nan'))
        spotFeatures['time_signature'].append(float('nan'))
        spotFeatures['energy'].append(float('nan'))
        
    # get the chord information
    uniqChords = dfAllSongs['Chords'][iSong].split(',')[1:]
    nUniqueChords.append(len(uniqChords))
    chordDensity.append(nUniqueChords[iSong]/spotFeatures['duration_m'][iSong])

    tmpHard = []
    for kCh in range(0, nUniqueChords[iSong]):
        tmpHard.append(uniqChords[kCh] not in standardList)

    nHardChords.append(sum(tmpHard))


In [28]:
# combine everything into one DF called allFeatures
spotFeatures = pd.DataFrame.from_dict(spotFeatures)
# spotFeatures.head()
allFeatures = spotFeatures

allFeatures['n_unique_chords']     = nUniqueChords
allFeatures['n_difficult_chords']  = nHardChords
allFeatures['chord_per_min']       = chordDensity

print(len(allFeatures))
allFeatures.head()

1099


,tempo,duration_m,time_signature,energy,n_unique_chords,n_difficult_chords,chord_per_min
0,89.132,2.408833,4.0,0.7260,7,1,2.905971
1,76.364,1.981967,4.0,0.0439,20,10,10.090987
2,129.012,4.212883,4.0,0.4900,2,0,0.474734
3,107.674,4.670450,3.0,0.2830,10,1,2.141121
4,122.361,3.128883,4.0,0.8200,13,8,4.154837


In [29]:
# append to the sql db
allFeatures.to_sql('features', con=conn, if_exists='append')


In [30]:
# cur.close()
# conn.close()

In [9]:
# load the model
import pickle
# load the model from disk
filename = 'lr_adv_model_0930.sav'
lr_adv = pickle.load(open(filename, 'rb'))

In [38]:
allFeat = pd.read_sql_query("SELECT * FROM features", conn) #get from sql
# allFeat.dropna(axis=0, how='any', inplace=True) # remove nans
allFeat.reset_index(drop=True, inplace=True) #reset index
allFeat.drop('index', axis=1, inplace=True) # drop the extra column sql added


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(allFeat)
feat_scale = scaler.transform(allFeat)

In [39]:
predictions = lr_adv.predict(feat_scale)

In [40]:
predictions

array([0, 0, 0, ..., 0, 1, 0], dtype=uint8)

In [41]:
label = pd.read_sql_query("SELECT Label FROM basic_info", conn)
label['model_isAdv']= predictions
label.rename(columns={'Label': 'ug_label'})

In [46]:
label.to_sql('label', con=conn, if_exists='append')